In [ ]:
# Import necessary libraries
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Function to load an image from the specified path
def load_image(image_path):
    try:
        # Load the image in its original format
        image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        if image is None:
            raise FileNotFoundError("Image not found or unsupported format.")
        return image
    except Exception as e:
        print(f"Error loading image: {e}")
        return None

# Function to convert an image to grayscale
def convert_to_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Function to convert an image to HSV
def convert_to_hsv(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Function to convert an image to CMYK 
def convert_to_cmyk(image):
    bgr_scaled = image / 255.0  # Scale to [0,1]
    k = 1 - np.max(bgr_scaled, axis=2)  # Calculate K channel
    c = (1 - bgr_scaled[..., 2] - k) / (1 - k + 1e-10)  # Avoid divide by zero
    m = (1 - bgr_scaled[..., 1] - k) / (1 - k + 1e-10)
    y = (1 - bgr_scaled[..., 0] - k) / (1 - k + 1e-10)
    cmyk = (np.stack((c, m, y, k), axis=2) * 255).astype(np.uint8)
    return cmyk

# Function to display images
def display_images(images_dict):
    plt.figure(figsize=(12, 8))
    for i, (title, image) in enumerate(images_dict.items()):
        plt.subplot(2, 3, i + 1)
        if len(image.shape) == 2:  # Grayscale image
            plt.imshow(image, cmap='gray')
        else:
            plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.title(title)
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Function to save the converted images
def save_image(image, save_path):
    try:
        cv2.imwrite(save_path, image)
        print(f"Image saved at {save_path}")
    except Exception as e:
        print(f"Error saving image: {e}")

if __name__ == "__main__":
    # Path to the image file
    image_path = "labPhoto.jpg"  

    # Load the image
    image = load_image(image_path)
    if image is not None:
        # Convert the image to different color models
        grayscale_image = convert_to_grayscale(image)
        hsv_image = convert_to_hsv(image)
        cmyk_image = convert_to_cmyk(image)
        
        # Display all images
        images_dict = {
            "Original Image": image,
            "Grayscale Image": grayscale_image,
            "HSV Image": hsv_image,
            "CMYK Image": cmyk_image[..., :3],  
        }
        display_images(images_dict)
        
        save_image(grayscale_image, "grayscale_labPhoto.jpg")
        save_image(hsv_image, "hsv_labPhoto.jpg")
        save_image(cmyk_image, "cmyk_labPhoto.tiff")

    else:
        print("Failed to load the image.")


## bonus 
def interactive_conversion():
   
    try:
        # Prompt the user for the image path and desired color model
        image_path = input("Enter the full path of the image: ").strip()
        color_model = input("Enter desired color model (grayscale/hsv/cmyk): ").strip().lower()

        # Print the user inputs
        print(f"User Input - Image Path: {image_path}, Color Model: {color_model}")

        image = load_image(image_path)
        if image is None:
            print("Failed to load the image. Please check the file path.")
            return

        if color_model == "grayscale":
            converted_image = convert_to_grayscale(image)
            save_path = "converted_grayscale_image.jpg"
        elif color_model == "hsv":
            converted_image = convert_to_hsv(image)
            save_path = "converted_hsv_image.jpg"
        elif color_model == "cmyk":
            converted_image = convert_to_cmyk(image)
            save_path = "converted_cmyk_image.tiff"
        else:
            print("Invalid color model. Please choose from grayscale, hsv, or cmyk.")
            return

        save_image(converted_image, save_path)
        print(f"The image has been successfully converted to {color_model} ")

        print("Displaying the converted image...")
        plt.figure(figsize=(6, 6))
        if color_model == "grayscale":
            plt.imshow(converted_image, cmap='gray')
        elif color_model == "hsv":
            plt.imshow(cv2.cvtColor(converted_image, cv2.COLOR_HSV2RGB))
        elif color_model == "cmyk":
            plt.imshow(converted_image[..., :3])  
        plt.title(f"Converted Image - {color_model.upper()}")
        plt.axis('off')
        plt.show()

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    interactive_conversion()

